In [8]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

class RsiOscillator(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    # Do as much initial computation as possible
    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)

    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000, commission=.002)
stats = bt.run()
bt.plot()




E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3057', ...)

In [9]:
import datetime
import pandas_ta as ta
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover,TrailingStrategy
# from backtesting.test import GOOG

data = pd.read_csv("1h_TATAPOWER.csv")

class RsiOscillator(TrailingStrategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14
    mysize = 1

    # Do as much initial computation as possible
    def init(self):
        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)
        super().init()
        super().set_trailing_sl(3)


    # Step through bars one by one
    # Note that multiple buys are a thing here
    def next(self):
        next_open = self.data.Close[-2]  # Next candle's open price
        
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            entry_price = next_open + 0.1  # Example: entry 1 cent above open
            sl = entry_price  -  50
            tp = entry_price +   50*3 #sl_atr * 4
            self.buy( sl=sl, tp=tp, size=self.mysize)

bt = Backtest(data, RsiOscillator, cash=10_000, commission=.002)
stats = bt.run()
bt.plot()


C:\Users\Admin\AppData\Local\Temp\ipykernel_15284\4279461448.py:39: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, RsiOscillator, cash=10_000, commission=.002)
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3414', ...)

In [3]:
data

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2023-01-02 09:15:00+05:30,208.750000,209.449997,208.199997,209.250000,209.250000,0
1,2023-01-02 10:15:00+05:30,209.350006,210.500000,209.250000,210.199997,210.199997,1211203
2,2023-01-02 11:15:00+05:30,210.250000,211.199997,209.800003,211.100006,211.100006,1038769
3,2023-01-02 12:15:00+05:30,211.149994,211.199997,210.199997,210.750000,210.750000,618475
4,2023-01-02 13:15:00+05:30,210.750000,211.149994,210.199997,211.100006,211.100006,620072
...,...,...,...,...,...,...,...
2431,2024-06-05 11:15:00+05:30,414.799988,416.049988,409.000000,415.549988,415.549988,3689887
2432,2024-06-05 12:15:00+05:30,415.450012,420.850006,413.799988,418.799988,418.799988,2256074
2433,2024-06-05 13:15:00+05:30,418.649994,424.000000,417.799988,421.500000,421.500000,2256791
2434,2024-06-05 14:15:00+05:30,421.750000,424.450012,419.850006,423.100006,423.100006,2236506


In [15]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd 
import numpy as np
from backtesting.test import SMA

data = pd.read_csv("1h_TATAPOWER.csv")
# data = pd.read_csv("D_RELIANCE1.csv")

# data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

data = data.dropna()
# data = pd.DataFrame()

# data= data[1800:2400]
class SmaCross(Strategy):
    n1 = 5 #10
    n2 = 10 #20
    mysize = 30
    sl_coef = 1

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):

         # get previous bar close
        price = self.data["Close"][-1]
    

        # long/short SL and TP
        long_sl = price * 0.98
        long_tp = price * 1.03
        short_sl = price * 1.02
        short_tp = price * 0.97

        # signal logic
        if crossover(self.sma1, self.sma2):
                self.buy(sl = long_sl, tp = long_tp,size=self.mysize)
        elif crossover(self.sma2, self.sma1):
            self.sell(sl = short_sl, tp = short_tp, size=self.mysize)

        # tsl
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, 
                            self.data["Close"][-1] * 0.98) 
            else: # short
                trade.sl = min(trade.sl or np.inf, 
                            self.data["Close"][-1] * 1.02) 

        
# bt = Backtest(GOOG, SmaCross,
#               cash=10000, commission=.002,
#               exclusive_orders=True)


bt = Backtest(data, SmaCross, cash=100000, commission=.02)
stats = bt.run()

print(stats)
bt.plot()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15284\1216903805.py:61: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, SmaCross, cash=100000, commission=.02)


Start                                     0.0
End                                    1487.0
Duration                               1487.0
Exposure Time [%]                   51.612903
Equity Final [$]                 79616.019916
Equity Peak [$]                      100000.0
Return [%]                          -20.38398
Buy & Hold Return [%]                 0.45295
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -20.39598
Avg. Drawdown [%]                   -20.39598
Max. Drawdown Duration                 1466.0
Avg. Drawdown Duration                 1466.0
# Trades                                 83.0
Win Rate [%]                         7.228916
Best Trade [%]                       1.020408
Worst Trade [%]                     -3.916827
Avg. Trade [%]                    

E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5069', ...)

In [5]:
data = pd.read_csv("D_RELIANCE1.csv")
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-02,2353.649170,2380.416260,2351.987793,2377.554932,2369.185791,2658087
1,2023-01-03,2367.540283,2374.878174,2351.618652,2360.156250,2351.848389,3829466
2,2023-01-04,2360.110107,2363.848389,2320.421143,2324.620850,2316.438232,4632445
3,2023-01-05,2329.189697,2341.096436,2311.191162,2320.467285,2312.299316,6818549
4,2023-01-06,2332.097168,2351.757080,2324.390137,2341.557861,2333.315430,3174798
...,...,...,...,...,...,...,...
344,2024-05-30,2871.000000,2880.800049,2840.199951,2849.699951,2849.699951,6603429
345,2024-05-31,2862.600098,2884.500000,2844.500000,2860.800049,2860.800049,15534916
346,2024-06-03,2966.000000,3029.000000,2918.000000,3020.649902,3020.649902,10763971
347,2024-06-04,2996.100098,2996.100098,2718.600098,2794.550049,2794.550049,18354549


In [4]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd 
import numpy as np
from backtesting.test import SMA

data = pd.read_csv("D_RELIANCE1.csv")
# data = pd.read_csv("D_RELIANCE1.csv")

# data['Datetime'] = pd.to_datetime(data['Datetime'])
# data.set_index('Datetime', inplace=True)

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

data = data.dropna()
# data = pd.DataFrame()

# data= data[1800:2400]
# data= data[1000:]
class SmaCross(Strategy):
    n1 = 5 #10
    n2 = 10 #20
    mysize = 20
    sl_coef = 1

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):

         # get previous bar close
        price = self.data["Close"][-1]
    

        # long/short SL and TP in percentage
        long_sl = price * 0.98
        long_tp = price * 1.03
        short_sl = price * 1.02
        short_tp = price * 0.97


        # signal logic
        if crossover(self.sma1, self.sma2):
                self.buy(sl = long_sl, tp = long_tp,size=self.mysize)
        elif crossover(self.sma2, self.sma1):
                self.sell(sl = short_sl, tp = short_tp, size=self.mysize)
        
        # tsl
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, 
                            self.data["Close"][-1] * 0.98) 
            else: # short
                trade.sl = min(trade.sl or np.inf, 
                            self.data["Close"][-1] * 1.02) 

        

bt = Backtest(data, SmaCross, cash=10000, commission=.001)
stats = bt.run()

print(stats)
bt.plot()

Start                     2023-01-02 00:00:00
End                       2024-06-05 00:00:00
Duration                    520 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               19.513537
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
E:\anaconda3\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1832', ...)